# Importing necessary packages

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import time
import matplotlib.pyplot as plt

# Loading Map dataset

In [5]:
map_df = gpd.read_file(r'neighbourhoods.geojson')


## Venues in Neighbourhood -- Go further down (you don't need to run this)

As part of our model, we are looking to explore proximity to certain venues as a possible price predictor. Walkability and ability to reach places maybe a deal-maker or breaker when it comes to choosing a accomodation. Proximity to certain venues, such as principal touristic attractions, restaurants, cafes and even shops could help us predict price. For this, we will use Foursquare API to explore the venues per neighbourhood. As discussed before, Old Town is the area which concentrates the majority of Airbnb listings.

We now retrieve the list of venues with their locations. We will use Foursquare API to explore the venues around the listings, using the latitude and longitude of each neighbourhood. We dill then find out which venues are the most common and select the most common venues as points of interest (POIS) for our accessibility analysis.

In [6]:
# Define Foursquare Credentials 

CLIENT_ID = 'P4KKN4V0L5GWJ0NJOIWAW0UAEFZ5IDUJWOLV3D3CDY4VHVSR'
CLIENT_SECRET =  'LIKA4WVFMBO1GHGO14PWKFQQZSTIRMHUB52BIL1Q5WRPPAYC'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P4KKN4V0L5GWJ0NJOIWAW0UAEFZ5IDUJWOLV3D3CDY4VHVSR
CLIENT_SECRET:LIKA4WVFMBO1GHGO14PWKFQQZSTIRMHUB52BIL1Q5WRPPAYC


In [7]:
map_df.head()

,neighbourhood,neighbourhood_group,geometry
0,Blankenfelde/Niederschönhausen,Pankow,"MULTIPOLYGON (((13.41191 52.61487, 13.41183 52..."
1,Helmholtzplatz,Pankow,"MULTIPOLYGON (((13.41405 52.54929, 13.41422 52..."
2,Wiesbadener Straße,Charlottenburg-Wilm.,"MULTIPOLYGON (((13.30748 52.46788, 13.30743 52..."
3,Schmöckwitz/Karolinenhof/Rauchfangswerder,Treptow - Köpenick,"MULTIPOLYGON (((13.70973 52.39630, 13.70926 52..."
4,Müggelheim,Treptow - Köpenick,"MULTIPOLYGON (((13.73762 52.40850, 13.73773 52..."


In [8]:
type(map_df.geometry)

geopandas.geoseries.GeoSeries

From geometry column in the GeoDataFrame, we can see that it returns a GeoSeries. To retrieve the venues per neighbourhood, we first extract the latitude and longitude or x and y from the point object in the GeoSeries. The following code returns x and y as separate GeoDataFrame columns within the original map_df.

In [9]:
map_df["longitude"] = map_df.centroid.x
map_df["latitude"] = map_df.centroid.y

In [10]:
map_df.head()

,neighbourhood,neighbourhood_group,geometry,longitude,latitude
0,Blankenfelde/Niederschönhausen,Pankow,"MULTIPOLYGON (((13.41191 52.61487, 13.41183 52...",13.400340,52.612536
1,Helmholtzplatz,Pankow,"MULTIPOLYGON (((13.41405 52.54929, 13.41422 52...",13.418908,52.543687
2,Wiesbadener Straße,Charlottenburg-Wilm.,"MULTIPOLYGON (((13.30748 52.46788, 13.30743 52...",13.311817,52.472878
3,Schmöckwitz/Karolinenhof/Rauchfangswerder,Treptow - Köpenick,"MULTIPOLYGON (((13.70973 52.39630, 13.70926 52...",13.654120,52.375075
4,Müggelheim,Treptow - Köpenick,"MULTIPOLYGON (((13.73762 52.40850, 13.73773 52...",13.683591,52.407020


In [11]:
import requests

In [12]:
# Function to loop for venues through all neighbourhoods

def getNearbyVenues(names, latitudes, longitudes, radius):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
         # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we run the above function on each neighborhood and create a new dataframe called edinburgh_venues.

In [13]:
berlin_venues = getNearbyVenues(names=map_df['neighbourhood'],
                                   latitudes=map_df['latitude'],
                                   longitudes=map_df['longitude'],
                                 radius = 500
                                  )

Blankenfelde/Niederschönhausen
Helmholtzplatz
Wiesbadener Straße
Schmöckwitz/Karolinenhof/Rauchfangswerder
Müggelheim
Biesdorf
Nord 1
West 5
Frankfurter Allee Nord
Buch
Kaulsdorf
None
None
nördliche Luisenstadt
Nord 2
Moabit West
Volkspark Wilmersdorf
Alexanderplatz
Brunnenstr. Süd
Wedding Zentrum
Moabit Ost
Parkviertel
Südliche Friedrichstadt
Karl-Marx-Allee-Nord
Tempelhofer Vorstadt
südliche Luisenstadt
Buchholz
Karow
Weißensee
Pankow Zentrum
Blankenburg/Heinersdorf/Märchenland
Schönholz/Wilhelmsruh/Rosenthal
Mierendorffplatz
Pankow Süd
Weißensee Ost
Prenzlauer Berg Nordwest
Prenzlauer Berg Nord
Charlottenburg Nord
Barstraße
Prenzlauer Berg Ost
Prenzlauer Berg Südwest
Prenzlauer Berg Süd
Heerstrasse
Schloß Charlottenburg
Neue Kantstraße
Schmargendorf
Düsseldorfer Straße
Brunsbütteler Damm
Heerstraße Nord
Forst Grunewald
Schloßstr.
Hakenfelde
Wilhelmstadt
Albrechtstr.
Siemensstadt
Gatow / Kladow
Teltower Damm
Drakestr.
Lichtenrade
Zehlendorf  Nord
Zehlendorf  Südwest
Friedenau
Alt  Tr

In [14]:
print(berlin_venues.shape)
berlin_venues.head()

(1779, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Helmholtzplatz,52.543687,13.418908,Goldhahn & Sampson,52.542200,13.420431,Gourmet Shop
1,Helmholtzplatz,52.543687,13.418908,Salt n Bone,52.544047,13.420102,Gastropub
2,Helmholtzplatz,52.543687,13.418908,Hum vegan cuisine,52.543651,13.416496,Vietnamese Restaurant
3,Helmholtzplatz,52.543687,13.418908,Wohnzimmer,52.543396,13.419288,Café
4,Helmholtzplatz,52.543687,13.418908,Kajumi,52.544094,13.421894,Coffee Shop


In [15]:
# Saving Data set
berlin_venues.to_csv(r'Berlin_Venues.csv')

## Importing File

In [7]:
# Read venues data set
berlin_venues = pd.read_csv('Berlin_Venues.csv', index_col=0)

In [8]:
berlin_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adlershof,8,8,8,8,8,8
Albrechtstr.,21,21,21,21,21,21
Alexanderplatz,30,30,30,30,30,30
Allende-Viertel,4,4,4,4,4,4
Alt Treptow,28,28,28,28,28,28
...,...,...,...,...,...,...
Wilhelmstadt,6,6,6,6,6,6
Zehlendorf Nord,6,6,6,6,6,6
Zehlendorf Südwest,10,10,10,10,10,10


## Unique Categories of venues

In [9]:
print('There are {} unique categories.'.format(len(berlin_venues['Venue Category'].unique())))

There are 277 unique categories.


## Number of Venues per Category

In [10]:
berlin_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,1,1,1,1,1,1
Adult Boutique,1,1,1,1,1,1
African Restaurant,3,3,3,3,3,3
Airport Service,1,1,1,1,1,1
American Restaurant,3,3,3,3,3,3
...,...,...,...,...,...,...
Wine Shop,10,10,10,10,10,10
Winery,1,1,1,1,1,1
Wings Joint,1,1,1,1,1,1


## Generating Dummies for each category 

In [11]:
# Generating dummies for different types of venues
berlin_dummies = pd.get_dummies(berlin_venues[['Venue Category']], prefix = "", prefix_sep = "")

In [12]:
## Add neighbourhood column back to df
berlin_dummies['Neighbourhood'] = berlin_venues['Neighbourhood']

In [13]:
# Move neighbourhood column to the first column
fixed_columns = [berlin_dummies.columns[-1]] + list(berlin_dummies.columns[:-1])
berlin_dummies = berlin_dummies[fixed_columns]

# display
berlin_dummies.head()

,Neighbourhood,ATM,Adult Boutique,African Restaurant,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo Exhibit
0,Helmholtzplatz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Helmholtzplatz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Helmholtzplatz,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Helmholtzplatz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Helmholtzplatz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# New df dimensions
berlin_dummies.shape

(1770, 278)

## Quick analysis of venues (mean and frequency)

In [15]:
berlin_grouped = berlin_dummies.groupby('Neighbourhood').mean().reset_index()
berlin_grouped

,Neighbourhood,ATM,Adult Boutique,African Restaurant,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Zoo Exhibit
0,Adlershof,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
1,Albrechtstr.,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,Alexanderplatz,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000,0.000000,0.033333,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
3,Allende-Viertel,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,Alt Treptow,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.035714,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,Wilhelmstadt,0.0,0.000000,0.0,0.0,0.0,0.0,0.166667,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
129,Zehlendorf Nord,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
130,Zehlendorf Südwest,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.100000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
131,nördliche Luisenstadt,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.066667,0.000000,...,0.0,0.000000,0.0,0.0,0.033333,0.0,0.0,0.0,0.033333,0.0


## Extracting each Neighbourhood with top 5 common venues

As you can see above due to the large number of venues and their different subcategories we will try to aggregate these across the top 5 venues.

In [16]:
num_top_venues = 5

for x in berlin_grouped['Neighbourhood']:
    print("----"+x+"----")
    temp = berlin_grouped[berlin_grouped['Neighbourhood'] == x].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adlershof----
               venue  freq
0   Greek Restaurant  0.25
1        Supermarket  0.25
2               Bank  0.12
3  Trattoria/Osteria  0.12
4      Shopping Mall  0.12


----Albrechtstr.----
                venue  freq
0         Supermarket  0.19
1            Bus Stop  0.14
2                 Gym  0.10
3  Italian Restaurant  0.05
4  Turkish Restaurant  0.05


----Alexanderplatz----
            venue  freq
0           Hotel  0.13
1  History Museum  0.10
2  Ice Cream Shop  0.07
3     Candy Store  0.03
4          Garden  0.03


----Allende-Viertel----
                  venue  freq
0              Bus Stop  0.25
1           Supermarket  0.25
2  Fast Food Restaurant  0.25
3          Soccer Field  0.25
4     Outdoor Sculpture  0.00


----Alt  Treptow----
                venue  freq
0                Café  0.11
1  Italian Restaurant  0.07
2            Bus Stop  0.07
3              Bakery  0.07
4            Platform  0.07


----Alt-Hohenschönhausen Nord----
            venue  freq
0  

                venue  freq
0           Drugstore  0.11
1  Rock Climbing Spot  0.11
2  Mexican Restaurant  0.11
3              Bakery  0.11
4   Electronics Store  0.11


----Hellersdorf-Ost----
                  venue  freq
0        Massage Studio  0.25
1           Supermarket  0.25
2      Sushi Restaurant  0.25
3  Gym / Fitness Center  0.25
4       Organic Grocery  0.00


----Hellersdorf-Süd----
            venue  freq
0     Supermarket   0.3
1           Trail   0.1
2  Discount Store   0.1
3   Metro Station   0.1
4   Shopping Mall   0.1


----Helmholtzplatz----
                 venue  freq
0                 Café  0.10
1                  Bar  0.10
2   Italian Restaurant  0.07
3          Coffee Shop  0.07
4  Japanese Restaurant  0.07


----Johannisthal----
              venue  freq
0      Dessert Shop  0.09
1               Pub  0.09
2      Burger Joint  0.09
3     Movie Theater  0.09
4  Sushi Restaurant  0.09


----Kantstraße----
                venue  freq
0  Italian Restaurant  0.07
1

                venue  freq
0  Italian Restaurant  0.10
1      Ice Cream Shop  0.10
2           Drugstore  0.10
3                Café  0.10
4           BBQ Joint  0.05


----Schmöckwitz/Karolinenhof/Rauchfangswerder----
                venue  freq
0  Italian Restaurant  0.25
1   German Restaurant  0.25
2     Harbor / Marina  0.25
3        Tram Station  0.25
4        Neighborhood  0.00


----Schöneberg-Nord----
                venue  freq
0                Café  0.13
1        Cocktail Bar  0.13
2  Italian Restaurant  0.07
3  Falafel Restaurant  0.07
4   Korean Restaurant  0.07


----Schöneberg-Süd----
                venue  freq
0  Italian Restaurant  0.07
1      Ice Cream Shop  0.07
2       Grocery Store  0.07
3    Doner Restaurant  0.07
4                Café  0.03


----Schönholz/Wilhelmsruh/Rosenthal----
         venue  freq
0     Bus Stop  0.25
1          Pub  0.25
2  Pizza Place  0.25
3       Bakery  0.25
4    Nightclub  0.00


----Siemensstadt----
                    venue  freq
0 

In [17]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
import numpy as np

# New dataframe ordered
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = berlin_grouped['Neighbourhood']

for ind in np.arange(berlin_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], 
                                                                          num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adlershof,Greek Restaurant,Supermarket,Trattoria/Osteria,Drugstore,Shopping Mall
1,Albrechtstr.,Supermarket,Bus Stop,Gym,Italian Restaurant,Automotive Shop
2,Alexanderplatz,Hotel,History Museum,Ice Cream Shop,Monument / Landmark,Spa
3,Allende-Viertel,Soccer Field,Fast Food Restaurant,Supermarket,Bus Stop,Event Space
4,Alt Treptow,Café,Italian Restaurant,Bus Stop,Platform,Bakery
...,...,...,...,...,...,...
128,Wilhelmstadt,Park,Halal Restaurant,Convenience Store,Supermarket,Argentinian Restaurant
129,Zehlendorf Nord,Coffee Shop,Greek Restaurant,Supermarket,Metro Station,Bus Stop
130,Zehlendorf Südwest,German Restaurant,Lake,Public Art,Art Gallery,School
131,nördliche Luisenstadt,German Restaurant,Italian Restaurant,Art Gallery,Bar,Trattoria/Osteria


In [19]:
print('There are {} unique categories.'.format(len(neighbourhoods_venues_sorted['1st Most Common Venue'].unique())))

There are 46 unique categories.


In [20]:
neighbourhoods_venues_sorted['1st Most Common Venue'].value_counts()

Supermarket                   22
Italian Restaurant            14
Café                          12
Bus Stop                       8
Bakery                         7
Hotel                          7
German Restaurant              7
Soccer Field                   5
Bar                            4
Cocktail Bar                   3
Tram Station                   3
Park                           3
Coffee Shop                    2
Nightclub                      2
Pub                            2
Restaurant                     2
Farmers Market                 1
Department Store               1
Art Gallery                    1
Bank                           1
Wine Bar                       1
Metro Station                  1
Construction & Landscaping     1
Pizza Place                    1
Pool                           1
Rental Car Location            1
Post Office                    1
Drugstore                      1
Insurance Office               1
Furniture / Home Store         1
Gas Statio

In [21]:
neighbourhoods_venues_sorted.groupby('1st Most Common Venue').count()

,Neighbourhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1st Most Common Venue,,,,,
Airport Service,1,1,1,1,1
Art Gallery,1,1,1,1,1
Bakery,7,7,7,7,7
Bank,1,1,1,1,1
Bar,4,4,4,4,4
Beach,1,1,1,1,1
Bookstore,1,1,1,1,1
Botanical Garden,1,1,1,1,1
Bus Stop,8,8,8,8,8


As you can see above the most common venues for this dataset are Supermarket, Hotel, German Restaurant, Cafe, Bus Stop, Bakery and Restaurant. Following these are bars, Nightclubs and drugstores. 

Further you can observe that numerous restaurants are in subcategories which makes them less frequent than if we would sum these together.

Another category we need to think about his hotels, since Airbnb are short-term rentals and they provide a different service/benefits than hotels do, there may be limited affect on price by this category 

Therefore, to have a more succinct analysis we will aggregate some categories and eliminate others

### Combining subcategories in the same variable

As you can see above there are numerous different categories for restaurants and bars so we will combine these to simplify our analysis and ensure that we do not overfit the model.

In [22]:
berlin_venues['Venue Category'].value_counts()

Supermarket            119
Café                   104
Italian Restaurant      74
Bakery                  58
German Restaurant       56
                      ... 
Szechuan Restaurant      1
Russian Restaurant       1
Dance Studio             1
Sports Club              1
Cycle Studio             1
Name: Venue Category, Length: 277, dtype: int64

In [23]:
#Get just Bar, Cafe And Restaurant
start_time = time.time()

# Significant venues already exists. Load from berlin_venues
df_sigven = berlin_venues.copy()
terms = ['Supermarket','Grocery Store', 'Bar', 'Pub', 'Restaurant', 'Café','Bakery','Coffee Shop']
df_sigven = df_sigven[df_sigven['Venue Category'].str.contains('|'.join(terms))]
df_sigven.to_csv(r'foursquare_amenities.csv', index=id, header=True)
method = 'loaded from CSV'

print('{:,} Signifcant Venues {} in {:,.2f} seconds'.format(len(df_sigven), method, time.time()-start_time))
df_sigven

877 Signifcant Venues loaded from CSV in 0.03 seconds


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Helmholtzplatz,52.543687,13.418908,Hum vegan cuisine,52.543651,13.416496,Vietnamese Restaurant
3,Helmholtzplatz,52.543687,13.418908,Wohnzimmer,52.543396,13.419288,Café
4,Helmholtzplatz,52.543687,13.418908,Kajumi,52.544094,13.421894,Coffee Shop
6,Helmholtzplatz,52.543687,13.418908,Pakolat,52.543814,13.416153,Coffee Shop
8,Helmholtzplatz,52.543687,13.418908,Pastificio Tosatti,52.543500,13.419621,Italian Restaurant
...,...,...,...,...,...,...,...
1752,Ost 1,52.568726,13.371505,Due Pini,52.568182,13.378672,Italian Restaurant
1755,Ost 1,52.568726,13.371505,PENNY,52.565090,13.370641,Supermarket
1757,Ost 2,52.573600,13.348588,Latichte,52.576094,13.348753,German Restaurant
1762,Ost 2,52.573600,13.348588,Provitrol Bäckerei,52.574776,13.354601,Bakery


In [24]:
# how many instances of each type of significant venue do we have 
pd.set_option('display.max_rows',500)
a = df_sigven['Venue Category'].value_counts()
a

Supermarket                      119
Café                             104
Italian Restaurant                74
Bakery                            58
German Restaurant                 56
Bar                               49
Coffee Shop                       36
Vietnamese Restaurant             28
Restaurant                        23
Cocktail Bar                      22
Chinese Restaurant                18
Greek Restaurant                  18
Doner Restaurant                  17
Sushi Restaurant                  17
Vegetarian / Vegan Restaurant     13
Pub                               12
Falafel Restaurant                11
Indian Restaurant                 11
Wine Bar                          11
Fast Food Restaurant              11
Japanese Restaurant               11
Korean Restaurant                 10
Turkish Restaurant                10
Asian Restaurant                   9
Middle Eastern Restaurant          9
Thai Restaurant                    8
Mediterranean Restaurant           7
G

Now we combine these subcategories intp 5 main categories to simplify our analysis

In [25]:
df_sigven.loc[df_sigven['Venue Category'].str.contains('Restaurant', case=False), 'Venue Category'] = 'Restaurant'
df_sigven.loc[df_sigven['Venue Category'].str.contains('Bar|Pub', case=False), 'Venue Category'] = 'Bar'
df_sigven.loc[df_sigven['Venue Category'].str.contains('Supermarket|Grocery Store', case=False), 'Venue Category'] = 'Supermarket'
df_sigven.loc[df_sigven['Venue Category'].str.contains('Café|Coffee Shop', case=False), 'Venue Category'] = 'Cafe'

#df_sigven.loc[df_sigven['Venue Category'].str.contains('Nightclub|Club', case=False), 'Venue Category'] = 'Club'
#I commented the above line out after I discovered there is just not enough data on Clubs in this dataset

df_sigven['Venue Category'].value_counts()

Restaurant     438
Cafe           140
Supermarket    126
Bar            115
Bakery          58
Name: Venue Category, dtype: int64

As it turns out there are only 17 venues fitting the description 'Club', this includes a sports club so we won't consider this as a significant venue, since we just do not have enough data for it (potential shortcoming)

# Merging Dataframe

Now we merge our significant venues dataframe with listings to have measure of how many of the 5 most common venues a listing has in its neighbourhood

In [26]:
df_sigven.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,Helmholtzplatz,52.543687,13.418908,Hum vegan cuisine,52.543651,13.416496,Restaurant
3,Helmholtzplatz,52.543687,13.418908,Wohnzimmer,52.543396,13.419288,Cafe
4,Helmholtzplatz,52.543687,13.418908,Kajumi,52.544094,13.421894,Cafe
6,Helmholtzplatz,52.543687,13.418908,Pakolat,52.543814,13.416153,Cafe
8,Helmholtzplatz,52.543687,13.418908,Pastificio Tosatti,52.543500,13.419621,Restaurant


In [27]:
#Determining how many of each of the 5 signifcant venues are in each neighbourhood
b = pd.crosstab(df_sigven['Neighbourhood'], df_sigven['Venue Category'])
b = b.reset_index()
b.tail()

Venue Category,Neighbourhood,Bakery,Bar,Cafe,Restaurant,Supermarket
112,Wilhelmstadt,0,0,0,2,1
113,Zehlendorf Nord,0,0,1,2,1
114,Zehlendorf Südwest,0,1,0,3,0
115,nördliche Luisenstadt,2,4,1,9,0
116,südliche Luisenstadt,0,5,7,7,0


In [28]:
#loading our listings cleaned dataset
df = pd.read_csv('listings_cleaned.csv', index_col=0)

#Removing unnecessary column -> should have done this way earlier but I missed the step
#df = df.drop(['neighbourhood_group_cleansed'],axis=1)
df['neighbourhood_cleansed'].head()

2015             Brunnenstr. Süd
2695    Prenzlauer Berg Nordwest
3176     Prenzlauer Berg Südwest
3309             Schöneberg-Nord
7071              Helmholtzplatz
Name: neighbourhood_cleansed, dtype: object

In [29]:
# Setting both columns have the same name for merging
df = df.rename(columns={'neighbourhood_cleansed': 'Neighbourhood'})
df = df.reset_index()
df = df.rename(columns={'index': 'id'})
df.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,Neighbourhood,neighbourhood_group_cleansed,property_type,...,smoking_allowed,nature_and_views,bedroom_necessities,time_since_first_review,time_since_last_review,bookings_per_month,occupancy_rate,occupancy_last_year,av_price_last_year,med_price_last_year
0,2015,2008-08-18,within an hour,90-99%,1.0,4.0,1.0,Brunnenstr. Süd,Mitte,Other,...,0.0,0.0,1.0,2-3 years,2-6 months,7.605000,1.000000,0.613699,81.900709,76.0
1,2695,2008-09-16,unknown,unknown,0.0,1.0,1.0,Prenzlauer Berg Nordwest,Pankow,Apartment,...,0.0,0.0,0.0,6-12 months,6-12 months,6.000000,0.394477,1.000000,NaN,NaN
2,3176,2008-10-19,within a day,100%,0.0,1.0,1.0,Prenzlauer Berg Südwest,Pankow,Apartment,...,0.0,0.0,0.0,4+ years,1+ year,0.490645,1.000000,0.397260,90.000000,90.0
3,3309,2008-11-07,unknown,unknown,0.0,1.0,1.0,Schöneberg-Nord,Tempelhof - Schöneberg,Apartment,...,0.0,0.0,1.0,4+ years,6-12 months,6.084000,1.000000,0.186301,27.131313,26.0
4,7071,2009-05-16,within an hour,100%,1.0,1.0,1.0,Helmholtzplatz,Pankow,Apartment,...,0.0,0.0,1.0,4+ years,2-6 months,15.210000,1.000000,0.928767,44.384615,44.0


In [30]:
# Merging by neighbourhoods
df_merged = pd.merge(df,b, on='Neighbourhood')

In [31]:
len(df_merged.index)

21934

In [32]:
df_merged.head()

,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,Neighbourhood,neighbourhood_group_cleansed,property_type,...,bookings_per_month,occupancy_rate,occupancy_last_year,av_price_last_year,med_price_last_year,Bakery,Bar,Cafe,Restaurant,Supermarket
0,2015,2008-08-18,within an hour,90-99%,1.0,4.0,1.0,Brunnenstr. Süd,Mitte,Other,...,7.605,1.0,0.613699,81.900709,76.0,0,3,3,6,1
1,22677,2010-03-02,within an hour,100%,1.0,1.0,0.0,Brunnenstr. Süd,Mitte,Apartment,...,10.140,1.0,0.375342,157.236842,160.0,0,3,3,6,1
2,37052,2010-06-21,within a day,100%,1.0,2.0,0.0,Brunnenstr. Süd,Mitte,Apartment,...,15.210,1.0,0.931507,47.560000,48.0,0,3,3,6,1
3,46172,2010-08-19,within a day,100%,0.0,2.0,1.0,Brunnenstr. Süd,Mitte,Other,...,0.507,1.0,0.276712,48.000000,48.0,0,3,3,6,1
4,57209,2010-10-27,within an hour,100%,0.0,1.0,0.0,Brunnenstr. Süd,Mitte,Apartment,...,10.140,1.0,0.276712,80.719697,80.0,0,3,3,6,1


In [30]:
# Save cleaned for modelling
df_merged.to_csv(r'listings_model.csv', header=True)